In [17]:
from autumn.projects.sm_covid2.common_school.project_maker import get_school_project_parameter_set
from autumn.projects.sm_covid2.common_school.calibration import get_bcm_object
import pandas as pd
from copy import copy
from pathlib import Path

In [18]:
bcm = get_bcm_object("FRA")

In [19]:
params_info = pd.read_excel("params_info.xlsx")

In [20]:
iso3 = "FRA"
first_date_with_death = 0
sero_age_min = 0
sero_age_max = 100
analysis='main'

In [21]:
model_params = get_school_project_parameter_set(
    iso3,
    first_date_with_death,
    sero_age_min,
    sero_age_max,
    analysis
).baseline

In [22]:
def get_param_val(p_name, model_params):
    if "." not in p_name:
        return model_params[p_name]
    else:
        nested_par = model_params
        nested_keys = p_name.split(".")
        for k in nested_keys:
            nested_par = nested_par[k]        
        return nested_par

In [23]:
def make_output_params_df(params_info, model_params):
    params_df = copy(params_info)

    for index in params_df.index:
        p_name = params_df.loc[index, 'param_key']
        value = params_df.loc[index, 'value']
        if value == "LOAD":
            if p_name in bcm.priors:
                p = bcm.priors[p_name]
                displayed_value = p.__repr__().replace("UniformPrior bounds:", "Uniform")
            else:
                displayed_value = get_param_val(p_name, model_params)
            params_df.loc[index, 'value'] = displayed_value
        elif value in ("HEADER", "SUBHEADER"):
            params_df.loc[index, 'value'] = ""
            params_df.loc[index, 'unit'] = ""
            params_df.loc[index, 'evidence'] = ""

            header_name = params_df.loc[index, 'legible_name']
            if value == "HEADER":
                params_df.loc[index, 'legible_name'] = f"BEGINBOLD{header_name}ENDBOLD"
            else:  # "SUBHEADER"
                params_df.loc[index, 'legible_name'] = f"BEGINITAL{header_name}ENDITAL"

        # tag calibrated parameters
        if p_name in bcm.priors:
            legible_name = params_df.loc[index, 'legible_name']
            params_df.loc[index, 'legible_name'] = f"{legible_name} SUPERC"

    return params_df

In [24]:
df = make_output_params_df(params_info, model_params)

In [25]:
pd.set_option('display.max_colwidth', None)
tex_table = df.to_latex(
    columns=['legible_name', 'value', 'evidence'],
    header=['Parameter', "Value/Distribution", "Evidence"],
    index=False,
    label="param_table",
    caption="Model parameters"
)

In [26]:
# Handle citations
tex_table = tex_table.replace("\\textbackslash cite\\", r"\cite").replace("\\}", "}")

In [27]:
# Equation mode not captured
tex_table = tex_table.replace("(\$m\_S\$)", r"($m_S$)")
tex_table = tex_table.replace("(\$m\_C\$)", r"($m_C$)")


In [28]:
# Use bold and italic font types for headings and sub-headings
tex_table = tex_table.replace("BEGINBOLD", r"\textbf{").replace("ENDBOLD", r"}")
tex_table = tex_table.replace("BEGINITAL", r"\quad \textit{").replace("ENDITAL", r"}")

In [29]:
# include superscript C for calibrated parameters
tex_table = tex_table.replace("SUPERC", r"\textsuperscript{C}")

In [30]:
# Change the table font so its width matches the pagewidth 
tex_table = tex_table.replace("\\begin{tabular}", r"\resizebox{\textwidth}{!}{\begin{tabular}")
tex_table = tex_table.replace("\\end{tabular}", r"\end{tabular}}")

In [31]:
out_path = Path.home() / "Models"/ "AuTuMN_new" / "docs" / "tex" / "tex_descriptions" / "projects" / "sm_covid" / "param_table.tex"

In [32]:
with out_path.open("w") as f:
    f.write(tex_table)